# Physics Informed Neural Networks (PINNs) for 1D Heat Equation

## 1. Introduction
In this workbook, we would be training a physics informed neural network model for 1D Heat equation. In specific, we will solve:

$\frac{\partial{u}}{\partial{t}} - \nu\nabla^{2} u(x) = f(x,t)$, where $x \in [-1,1]$, and $t \in (0,1]$


We will use the concept of PINNs, where the input to the neural network is $x, t$ and the output of the network is $u(x,t)$. Loss function is evaluated by enforcing the PDE and the boundary conditions.

Physics informed neural networks is made of a dense neural network that takes in the $(x,t)$ points in the domain and learns the physics from it using PDEs such as the one just above. 

The architecture of the network looks something like this:

![](https://www.researchgate.net/profile/Zhen-Li-105/publication/335990167/figure/fig1/AS:806502679982080@1569296631121/Schematic-of-a-physics-informed-neural-network-PINN-where-the-loss-function-of-PINN.ppm)

We will use the concept of PINNs, where the input to the neural network is $(x, t)$ and the output of the network is $u(x,t)$. Loss function is evaluated by enforcing the PDE and the boundary conditions.

We will begin the workbook with few imports and creating some helper functions

## 2. Imports and helper functions

* **deepxde**    - library for solving forward and inverse problems using PINNs
* **numpy**      - library for working multi-dimensional arrays and matrices
* **matplotlib** - library for data visualization

In [20]:
import os
import deepxde as dde
import numpy as np
# Backend tensorflow.compat.v1 or tensorflow
from deepxde.backend import tf
import time 
import matplotlib.pyplot as plt
t0 = time.time()

# Suppress warnings
import warnings
warnings.filterwarnings("ignore")

import mlflow
import shutil
import tempfile
import mlflow.tensorflow
import pandas as pd
import tensorflow as tf
from mlflow import pyfunc

# Setting a tracking uri to log the mlflow logs in a particular location tracked by 
from mlflow.tracking import MlflowClient
tracking_uri = os.environ.get("TRACKING_URL")
client = MlflowClient(tracking_uri=tracking_uri)
mlflow.set_tracking_uri(tracking_uri)

# Setting a tracking project experiment name to keep the experiments organized
experiments = client.list_experiments()
experiment_names = []
for exp in experiments:
    experiment_names.append(exp.name)
experiment_name = "pinn_deployment"
if experiment_name not in experiment_names:
    mlflow.create_experiment(experiment_name)
mlflow.set_experiment(experiment_name)
mlflow.tensorflow.autolog()

## Helper function to plot $u(x,t)$

In [21]:
def plot(geom_time,data,name):
    plt.figure()
    plt.scatter(geom_time[:,0], geom_time[:,1],s = 20, c=np.reshape(data, (len(data),)), cmap='jet')
    plt.colorbar()
    plt.xlabel('x domain')
    plt.ylabel('time domain')
    plt.title(name)
    plt.show()

## Functions used to calcuate neural network loss

PDE: $pde(x,y) : \frac{\partial{u}}{\partial{t}} - \nu\nabla^{2} u(x) =0$, where $\nu = 0.3$ 

Boundary conditions $func(x): \sin(\pi x)e^{x}$

Deepxde provides help functions to compute first and second derivatives

_dde.grad.jacobian_ is used to compute Jacobian matrix $J: J[i][j] = \frac{dy_i}{dx_j}$, where $i,j=0,…,dim_x-1$

_dde.grad.Hessian_ is used to compute Hessian matrix $H: H[i][j] = \frac{d^2y}{dx_i dx_j}$, where $i,j=0,…,dim_x-1$

In [22]:
# define the PDE for 1D Heat equations
def pde(x, y):
    dy_t = dde.grad.jacobian(y, x, j=1)
    dy_xx = dde.grad.hessian(y, x, j=0)

    return (
        dy_t
        - dy_xx*0.3)
    

def func(x):
    return np.sin(np.pi * x[:, 0:1]) * np.exp(-x[:, 1:])

## 2. initialization
Define geometry and time horizons for simulation

In [23]:
geom = dde.geometry.Interval(-1, 1)
timedomain = dde.geometry.TimeDomain(0, 1)
geomtime = dde.geometry.GeometryXTime(geom, timedomain)

Define boundary conditions and initial conditions

In [24]:
bc = dde.DirichletBC(geomtime, func, lambda _, on_boundary: on_boundary)
ic = dde.IC(geomtime, func, lambda _, on_initial: on_initial)

Create data points needed for building the model

In [25]:
data = dde.data.TimePDE(
    geomtime,
    pde,
    [bc, ic],
    num_domain=4000,
    num_boundary=2000,
    num_initial=1000,
    solution=func,
    num_test=1000,
)

### Steps in building the model 

- Define the number of layers and neurons per layer in the neural network
- Define activation function on these layers _tanh_
- Define function for initial weights for training _Glorot uniform_
- Train the network using Adam optimizer with learning rate of 0.001

In [26]:
layer_size = [2] + [32] * 3 + [1]

activation = "tanh"
initializer = "Glorot uniform"

net = dde.maps.FNN(layer_size, activation, initializer)
model = dde.Model(data, net)
model.compile("adam", lr=0.001, metrics=["l2 relative error"])

Compiling model...
Building feed-forward neural network...
'build' took 0.047724 s

'compile' took 0.390208 s



## 3. Training


In [ ]:
t1 = time.time()

losshistory, train_state = model.train(epochs=3000)

t2 = time.time()
print("training time:",(t2-t1))

Initializing variables...
Training model...

Step      Train loss                        Test loss                         Test metric   
0         [1.13e-01, 9.24e-02, 6.92e-01]    [1.11e-01, 9.24e-02, 6.92e-01]    [1.32e+00]    


## 4. Post-training Visualization

In [ ]:
model.save('./pinn_heat1d', protocol='tf.train.Saver', verbose=0)
dde.postprocessing.plot_loss_history(losshistory)
plt.show()


x_data = np.linspace(-1,1,num = 100)
t_data = np.linspace(0,1,num = 100)
test_x , test_t = np.meshgrid(x_data, t_data)
test_domain = np.vstack((np.ravel(test_x), np.ravel(test_t))).T
predicted_solution = model.predict(test_domain)
residual = model.predict(test_domain,operator=pde)



plot(test_domain,predicted_solution,"predicted solution")


plot(test_domain,residual,"residual")


print("total time")
print(t2-t0)